In [58]:
import pandas as pd
import os
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
import math as mt
from math import log
import itertools
import random

In [59]:
data_path = ''

In [60]:
df_edges = pd.read_csv(os.path.join(data_path,'1990_2000_edges.csv'))

In [61]:
df_filtered_authorships = pd.read_csv(os.path.join(data_path,'1990_2000_filtered_authorships.csv'))

In [62]:
df_nodes_catalogue = pd.read_csv(os.path.join(data_path,'nodes_catalogue.csv'))

In [63]:
df_nodes_catalogue.shape

(93912, 2)

In [64]:
G = nx.Graph()


In [65]:
G.add_nodes_from(df_nodes_catalogue.author.values)

In [66]:
G.add_edges_from(df_edges.values)

In [67]:
#G.nodes

In [68]:
#G.edges

In [69]:
def nodes_connected(u, v):
    return u in G.neighbors(v)

In [70]:
N = 160000

In [71]:
tasa_positivos = 1/2

In [72]:
positivos = round(N*(tasa_positivos))
negativos = round(N*(1-tasa_positivos))

In [73]:
positivos,negativos

(80000, 80000)

In [74]:
lista_no_conectados= []
no_conectados = 0
while no_conectados<negativos:
    a = b = ''
    while a == b:
        tuple_ = df_nodes_catalogue.author.sample(2).values
        a = tuple_[0]
        b = tuple_[1]

    #están conectados?
    con = nodes_connected(a, b)
    if con==False:
        lista_no_conectados.append([a,b])
        no_conectados+=1
    

In [75]:
no_conectados = pd.DataFrame(lista_no_conectados).rename(columns={0:'source',1:'target'})

In [76]:
conectados = df_edges.sample(positivos)

In [77]:
conectados['connected'] = 1

In [78]:
X = conectados.append(no_conectados).fillna(0)

In [79]:
X.connected.value_counts()

0.0    80000
1.0    80000
Name: connected, dtype: int64

In [80]:
X.drop_duplicates()

,source,target,connected
302112,Ioannis M. Rekleitis,Gregory Dudek,1.0
134258,Oscar Gustafsson,Lars Wanhammar,1.0
275475,Mark A. Franklin,Tienyo Pan,1.0
313452,Judith Baur,Doug Arnold,1.0
584357,Jürgen Weese,Derek L. G. Hill,1.0
...,...,...,...
79995,Roger Kenton Took,Aarno Lehtola,0.0
79996,Norbert Ascheuer,Deborah F. Cook,0.0
79997,René Mandiau,Jonathan K. Su,0.0
79998,Gregg T. Vesonder,Ok-Bae Chang,0.0


In [81]:
X.to_pickle('sample_wo_features.links_prediction')

In [82]:
sample = X.drop_duplicates().reset_index(drop=True).copy()

In [83]:
sample.connected.value_counts(1)

0.0    0.535791
1.0    0.464209
Name: connected, dtype: float64

# Pipeline

In [84]:
dict_count_papers = df_nodes_catalogue.set_index('author').to_dict()['id_article']

## Aggregated Features

### Sum of papers

In [85]:
sample['sum_of_papers'] = sample[['source','target']].apply(lambda x: dict_count_papers[x[0]]+dict_count_papers[x[1]], axis=1)

In [86]:
sample.head()

,source,target,connected,sum_of_papers
0,Ioannis M. Rekleitis,Gregory Dudek,1.0,55
1,Oscar Gustafsson,Lars Wanhammar,1.0,17
2,Mark A. Franklin,Tienyo Pan,1.0,22
3,Judith Baur,Doug Arnold,1.0,15
4,Jürgen Weese,Derek L. G. Hill,1.0,72


### Sum of neighbors

In [87]:
sample['sum_of_neighbors'] =sample[['source','target']].apply(lambda x: len([x for x in G.neighbors(x[0])]) + len([x for x in G.neighbors(x[1])]), axis=1)

In [88]:
#sample['sum_of_neighbors'].value_counts()

In [89]:
#sample[sample['sum_of_neighbors']==7]

### Sum of log (Secundary Neighbors count)

In [90]:
#l = [G.neighbors(y) for y in [x for x in G.neighbors('Alberto L. Sangiovanni-Vincentelli')]]

In [91]:
#[x for x in G.neighbors('Alberto L. Sangiovanni-Vincentelli')]

In [92]:
def extract_sec_neig(node):
    try:
        l = [G.neighbors(y) for y in [x for x in G.neighbors(node)]]
        secondary_neighbors = []
        for x in l:
            secondary_neighbors.extend([z for z in x])
        if 0==len(secondary_neighbors):
            return 1

        return len(secondary_neighbors)
        #return len(set(secondary_neighbors))
    except Exception as e:
        print(node,e)

In [93]:
def sum_sec_neig(node_a,node_b):
    try:
        return log(extract_sec_neig(node_a))+log(extract_sec_neig(node_b))
    except Exception as e:
        print(e)
        print(node_a,'_',node_b)
    

In [94]:
sample['log_secundary_neighbors']= sample[['source','target']].apply(lambda x: sum_sec_neig(x[0],x[1]), axis=1,)

### Shortest Distance

In [95]:
def extract_lenght_short_path(list_nodes):
    node_a,node_b = list_nodes[0],list_nodes[1]
    try:
        return nx.single_source_shortest_path_length(G,node_a)[node_b]
    except Exception as e:
        print(e)
        return np.nan        

In [96]:
from multiprocessing import Pool
p = Pool()

'Grant Lewison'
'Stanley Lawrence Marple Jr.'
'John E. Newhagen'
'Masatoshi Shiino'
'Pam Binns'
'Lishing Liu'
'Pei Hsia'
'Yoichi Tsuji'
'Perfecto Mariño'
'Mark D. Austin'
'Richard W. Eglese'
'Arnon Rungsawang'
'Ramón García-Domenech'
'Henry Cox'
'Robert D. Pates'
'Hanna Parnas'
'Luke Wildman'
'Jin Tian 0001'
'Alberto Malliani'
'Ignacio Aedo'
'Lukas Rosenthaler'
'Yu-Ru Syau'
'Milena M. Didic'
'Bernardino Arcay Varela'
'Joseph R. Litko'
'Mark A. Halstead'
'Jiri Dvorský'
'David R. Lester'
'Haidong Wu'
'Athar B. Tayyab'
'Ken Gonzales-Day'
'Yingbo Hua'
'Marianne Barcilon'
'James E. Hunton'
'Ren-ji Tao'
'James E. Hicks'
'Hongde Hu'
'Thomas M. Krummel'
'Humayun Akhtar'
'Jose Maria Turull Torres'
'Bozena Mannova'
'Eugene W. Stark'
'Brian M. Johnston'
'Ted J. Cooper'
'Sauro Longhi'
'Hari Gopal Shrestha'
'Peter-Pike J. Sloan'
'Mitsunori Mizumachi'
'Clark B. Archer'
'Simon L. Peyton Jones'
'Daniel D. McCracken'
'Steven J. Ross'
'Junichi Iljima'
'M. P. Biswal'
'Christian Krattenthaler'
'Cengiz Kah

'Mikko T. Siponen'
'Rolf Drechsler'
'Georg Rock'
'Nikos E. Mastorakis'
'Geoffrey Rutledge'
'Antti Autere'
'Gilberto Marzano'
'Max B. Reid'
'T. K. Dutta'
'Jin-Ru Chen'
'Lynn Ling X. Li'
'Reem Bahgat'
'Martin Schumacher'
'Jörg Caumanns'
'Michael J. Ohletz'
'R. Cooper'
'Jörg Raisch'
'Howell O. Istance'
'Gregory F. Cooper'
'Panu Raatikainen'
'Robert B. Grady'
'Michael J. Swain'
'Elena Lodi'
'A. Camussi'
'Michel Hébert'
'R. Mukundan'
'J. G. Dai'
'Maria Kwasnik'
'Djordje Boskov'
'Claus Rick'
'Henri Cohen'
'Ralph Gautzsch'
'Aniruddha Datta'
'Maria Paula Queluz'
'Dentcho N. Batanov'
'Martin Baca'
'Phillip Barry'
'Ronald Peikert'
'Fernando Pérez-González'
'Gang-Len Chang'
'Margarida Vaz Pato'
'Mansour Jamzad'
'Tianzi Jiang'
'M. M. Fonkam'
'Douglas Kunda'
'David Mead'
'Chris Metcalf'
'Hiroshi Shibasaki'
'Siamak A. Ardekani'
'Patric R. J. Östergård'
'Helmut Steckenbiller'
'Thiagarajan Balachander'
'Gabriel Soum'
'Gu Xunrang'
'Han-Seung Jung'
'Mauro Cesar Zanella'
'Shozo Kondo'
'Seiichi Uchida'
'A

'Juan W. Dixon''Wei-Ming Hu'

'Mark P. Grinnell'
'Horst Alzer'
'Carlos Avendaño'
'Julie J. McGowan'
'Zoltan J. Cendes'
'Ewald Lang'
'David J. McConnell'
'Maciej Szymanski'
'Jan B. Wilstrup'
'Stefan Voß'
'Francis Suraweera'
'Jiali Gao'
'Victor Carreño'
'Wieb Bosma'
'Terence P. Rout'
'Vojko Pahor'
'John A. Canaris'
'Palitha Dassanayake'
'Edward H. Freeman'
'Spyros Sioutas'
'Damu Radhakrishnan'
'Abdel-Karim S. O. Hassan'
'Koen L. Vincken'
'Manfred Pinkal'
'Bennett L. Fox'
'Mario Radovan'
'Xiaowen Li'
'Paul G. Bamberg'
'Ioannis Papaefstathiou'
'Xiao Chen'
'Andrea Bacciotti'
'Shigeki Goto'
'Monica Adya'
'Andreas Steiner'
'Churn-Jung Liau'
'Joseph M. Mellichamp'
'Evan E. Davidson'
'Radu Mateescu 0001'
'Bhu Dev Sharma'
'Christiane Bercoff'
'F. H. D. van Batenburg'
'Mohan K. Kadalbajoo'
'Hirotaka Nakayama'
'Masahiro Fukuda'
'Larry D. Rubin'
'Martin Hofmann 0001'
'Wolfgang Pohl'
'Ulf J. Timm'
'K. S. Murray'
'Howard A. Levine'
'Demetrios Fakinos'
'Paolo M. Camerini'
'Yuan Chen'
'Yu-Kai Chen'
'Ad

'Nils Gustafsson'
'Harald Bina'
'Edgar Körner'
'Axel Ruhe'
'Carles Rafels'
'John Shackell'
'Fujiki Morii'
'Christopher W. V. Hogue'
'Wojciech Maly'
'Yiannakis Sazeides'
'Paul J. Craven'
'Larry Spitz'
'Kyoung-Sig Roh'
'Kevin M. Brooks'
'Paul Chapman'
'G. H. Khachatrian'
'James Trevelyan'
'Gordon S. Novak Jr.'
'Jan T. Kim'
'Teruo Negishi'
'Jonathan R. Corney'
'A. Richard Newton'
'Brian S. Garra'
'Gilles Bertrand 0001'
'Tom Wesley'
'Graeme Simsion'
'Robert E. Schapire'
'Antonio Boccalatte'
'Yu Jin Zhang'
'Ching-Hsue Cheng'
'John Tillquist'
'I. G. Mitchell'
'Jong C. Park''Paul D. Henri'

'Natalie R. Spooner'
'Paul Fisher'
'Øistein Bøe'
'Arnd Steinmetz'
'Li Lee'
'Tze Leung Lai'
'Jan F. Broenink'
'Sean M. Callahan'
'Claes Wohlin'
'Ute Claussen'
'Barbara A. Burns'
'José Javier Dolado'
'Shian-Tang Tzeng'
'H. L. Seet'
'Michael Korkin'
'B. G. Prakash'
'Charles André'
'Albertus C. den Brinker'
'Dzung L. Pham''Bernhard Scholz'

'Trond Mannseth'
'Pilar Martínez Ortigosa'
'H. Paul Greenough'
'Heli K

'Rosann Webb Collins'
'Ioana Rus'
'Se-Young Oh'
'Luciano Lopez'
'Bin Zheng 0001'
'Stephen R. Parker'
'Niels Jørgen Christensen''Tim Hendtlass'

'Christopher D. Bryant''X. Zhou'

'Laszlo Monostori'
'Kannan Govindan'
'Harry W. Agius'
'Richard Kenner'
'Lennart Lövstrand'
'Ryuichi Yokogawa'
'D. Dotti'
'Jacques Blanc-Talon'
'Philip Leith'
'A. M. Clarke'
'Paulo E. Miyagi'
'Y.-M. Deng'
'Shigenori Ito'
'Gert-Martin Greuel'
'Margaret Martinez'
'Michael Koch 0001'
'Beatriz H. S. C. Rocha'
'Robert L. Devaney'
'Leslie E. Trotter Jr.'
'Chad Childers'
'William Z. Zahavi'
'Albert B. Short'
'Soe-Tsyr Yuan'
"Shin'ichi Koike"
'Grigore Rosu'
'David J. Milham'
'Damian J. Sheehy'
'Sandra R. Thuel'
'Bas H. P. J. Vermeer'
'Jaime Muñoz Masqué''Siamak Mohammadi'

'David B. Stewart'
'Mark Ramras'
'Michael K. Molloy''Dimitrios Kokkinakis'

'Konstantinos P. Triantis'
'Abani K. Patra'
'I. N. Kar'
'Tsu-Chin Tsao'
'Johannes Wallner'
'Bertrand T. David'
'Agostino G. Bruzzone'
'Tat-Ming Lok'
'Rajani R. Joshi'
'Petr So

...

'Anton Kotzig'
'Paul Bassett''Tor Steinar Schei'

'David W. Coit'
'José Luis Melús-Moreno'
'Jerome Lemaire'
'Markus Schwehm'
'Milena M. Head'
'Hon Keung Kwan'
'Xiao-Ping Wang'
'John P. Georges'
'Haijian Sun'
'Stephen W. L. Yip'
'Ansgar Schleicher''Mihai Nadin'

'Ashutosh Garg'
'Béla Bollobás''Mads Haahr'

'Marshall Walker''Timothy F. Cleghorn'

'Takao Ohuchi''Hugues Benoit-Cattin'

'Albertus S. J. Helberg'
'Capers Jones'
'Andrew L. Brook'
'Shigeomi Hara'
'Eduardo Asbun''Axel Hunger'
'D. Randall Wilson''Fernand Badano'


'Frank J. Ingrassia'
'Chang-Yeong Kim'
'L. S. Tang'
'Jordi Castro'
'James E. Molini'
'David M. Harvey'
'Gregory B. Newby'
'Placid M. Ferreira'
'M. Selim Akturk'
'Sauro Tulipani'
'Evan Torrie'
'Eva Cerezo'
'Michael Hamman'
'Clark Robinson'
'Stephan Olariu'
'Jean Dansereau'
'Carlo S. Regazzoni'
'Ana Carolina Salgado'
'Subhash Saini'
'Denis Bouyssou'
'Oliver Linssen'
'Chrysostomos D. Stylios'
'Damith J. Mudugamuwa'
'Hung-Yang Chang'
'Neil McIntosh'
'Alexander V. Voinov'
'B

'Andrew M. Drinnan'
'Norman Paskin'
'Myke Gluck'
'Dawn E. Holmes'
'B. Robert Helm'
'Sheila Corrall'
'Margreet Nool'
'David Cohn'
'Dolores Pérez-Bendito'
'Jacob O. Wobbrock'
'Arun Rao'
'Harold Ossher'
'Eric Spano'
'Thomas Stöckel'
'Arnaud Venet'
'Michael Wheeler'
'J. D. Becker'
'Andrzej J. Kasinski'
'Jouke C. Verlinden''Pravin M. Vaidya'

'Bongki Moon'
'Fe Zinnia Albay'
'Hermann Hild'
'Robert S. Arnold'
'Diane Rowland'
'Ernst W. Biersack'
'Burkard Polster''Gary A. Montague'

'Michael Kupferschmid'
'Gennady S. Cymbalyuk'
'Xinmin Lü'
'San-Yuan Wang'
'Joaquim Salvi''Urs E. Gattiker''Ismo Hakala'


'Carlos Angel Iglesias'
'Kiyohiko Nakamura'
'Gaetano Zanghirati'
'Wolfgang Kilian'
'Jacky Swan''Douglas H. Summerville'

'Gilbert Caplain'
'Dariusz Kania'
'Fang-Hsuan Cheng'
'Kai Hakkarainen'
'Bogdan Smolka'
'Mario Pennisi'
'Martin D. Carroll''Hans-Henrik Bjørset'

'Nell B. Dale'
'Carmelo Malta'
'T. Schmidt''Martin Hulin'

'Branislav R. Boricic'
'Aaron Heller'
'Vicente A. Mut''Jim Schimert'

'Kuo

'Barbara O. Rothbaum''Liang Chen 0008'

'Li Zhang 0001'
'Jarmo J. Ahonen'
'Ernst Lippe'
'George Chin Jr.''Werner M. Kistler'

'Ernst Althaus'
'Yurij S. Kharin'
'Jan-Roelof de Pijper'
'Valentina Markova''Charles W. Bostian'

'Ilker Ersoy'
'Soemon Takakuwa'
'Vitaly Maiorov'
'H. Park''Hironobu Niijima'

'Misha Koshelev'
'Clifford Brunk'
'Annemie Vorstermans'
'Khurram Muhammad'
'Amparo Fúster-Sabater'
'James I. Garrels'
'Robert G. Willhoft''Julie J. McGowan'

'Kirsten Heiler'
'Deren Wang'
'Chang-Jun Lin'
'Niels Grønbech-Jensen'
'Limin Xiang'
'Ferran Ruiz i Tarragó'
'D. Johnson''Alan P. Parkes'

'Rainer Pielot'
'Ramón Rizo'
'Yoshinori Ogata'
'Velio Tralli'
'Jim Rosenberg''James Ting-Ho Lo'

'Tomasz Borzyszkowski'
'Ramaswamy Nilakantan'
'Leif Aarthun Ims'
'Harald Schaschinger'
'Terri J. Lydiard'
'Monique E. van Donzel''Loula Fezoui'

'Hans-Peter Volz'
'Fulya Altiparmak'
'Babak A. Farshchian'
"V. Kondrat'ev"
'Hugo Tschirky'
'Patrick Chiu'
'Hideo Osawa'
'M. Isabel Acevedo Sotoca'
'Martin Rich'

'Vladlen S. Skvortsov'
'Shawmin Lei''Edward A. Rietman'

'Mathias Rautenberg'
'Gregory Shpitalnik'
'Spiros Ventouras'
'Ian Stroud'
'Christian P. Ullrich'
'Jirina Vejnarová''Jano I. van Hemert'

'Weibing Gao'
'Tim Haulick'
'Gregory Stainhaouer'
'Patrick Sallé'
'Alexander E. Mohr''R. Balakrishnan'

'Peter A. W. Lewis''Fernando Matía'

'Raymond H. Myers'
'Daniel Campi'
'David Benzie'
'Jordi Alvarez'
'Kenneth M. Wallquist''J. Michael Bennett'

'Francis LePage'
'Jun-Ho Oh'
'Miroslav Popovic'
'Jorma Taramaa'
'Werner Dubitzky'
'Jason B. Scholz'
'Hilary McLellan'
'Hajime Mochizuki'
'Bernard E. M. Jones'
'C. S. Sung'
'Christian Blumenröhr'
'Timothy S. Gegg-Harrison'
'Emile Fiesler'
'Jim White'
'Marcel Paul Schützenberger'
'John D. Valois'
'Hans P. Muhlfeld'
'Hans-Rüdiger Pfister'
'Eric Magnusson'
'Frank Stenger'
'Jens-Uwe Möller'
'Matthew Andrews''D. G. T. Thomas'

'Kiyoshi Nosu'
'Sadashiv Adiga'
'Petra Steffens'
'Mikael Jern''Frédéric Boyer'

'Richard G. Mathieu'
'David G. Novick'
'Ian Galton'

'Stefan Voß'
"Vicki L. O'Day"
'Neal Koblitz'
'Volker Claus'
'Tilman P. Otto'
'Dharma Thiruvaiyaru'
'Hiroyuki Yoshikawa'
'Antonio Frangioni'
'John W. Lamp'
'Chuck Hoover'
'Kazuya Sato'
'Birger Hjørland'
'Chenho Kung'
'Diganta Goswami'
'Wen Lea Pearn'
'Marie-Hélène Stefanini'
'Parviz Yegani'
'Ronda Hauben'
'Bruno Apolloni'
'Po L. Tien'
'Peter G. Trotter'
'François Pottier'
'Filipe M. Silva'
'Xavier Cousin'
'Richard Saeks'
'Cornelis Jan Kikkert'
'Isaac W. S. Wong'
'Helena B. Minervina'
'Gary L. Wise'
'Evan L. Porteus'
'Panagiotis Takis Metaxas'
'Pierre Bonton'
'Tim Bergin'
'Robert L. Baber'
'Maurice Aburdene'
'Shiuh-Pyng Shieh'
'Mark Berman''Michel Zasadzinski'

'Alexander M. Taratorin'
'Mario Rossi'
'Zhanna A. Chernyak'
'Shun-Jee Liu'
'Mike Williamson'
'Sylviane R. Schwer'
'U. Ranft'
'Thomas Ringler'
'Ming-Kang Liu'
'Arieh Warshel'
'María-Victoria Belmonte'
'San-qi Li'
'Ken Naono'
'F. Allard'
'Andrea Molinari'
'Andy Clark'
'Paul Fletcher'
'Andreas Faldum'
'Medha Pandit'
'John D. Farrara'

In [97]:
shortest_paths = p.map(extract_lenght_short_path,sample[['source','target']].values)

In [98]:
len(shortest_paths)

149312

In [99]:
sample['lenght_short_path'] = shortest_paths

In [100]:
sample['lenght_short_path'].value_counts()

1.0     69312
7.0     17961
6.0     17088
8.0     11038
5.0      8156
9.0      4867
4.0      2129
10.0     1742
11.0      600
3.0       353
12.0      182
2.0        42
13.0       39
14.0       19
15.0        3
17.0        1
Name: lenght_short_path, dtype: int64

In [101]:
#sample[['source','target']].tail().values

In [102]:
#sample[['source','target']].tail().apply(lambda x: extract_lenght_short_path(x[0],x[1]), axis=1)

In [103]:
#sample['lenght_short_path'] = sample[['source','target']].apply(lambda x: extract_lenght_short_path(x[0],x[1]), axis=1)

In [104]:
sample[sample.sum_of_neighbors==4].iloc[0].target

'Tor P. Schultz'

In [105]:
sample[sample.sum_of_neighbors==4].iloc[0].source

'Harry P. Schultz'

### Clustering index

In [106]:
def clustering_index(node_a,node_b):
    d = nx.clustering(G, nodes=[node_a,node_b])
    return sum(d.values())
    
    

In [107]:
sample['clustering_index_sum'] = sample[['source','target']].apply(lambda x: clustering_index(x[0],x[1]), axis=1)

In [108]:
import datetime

In [109]:
today = datetime.datetime.today()

In [110]:
today.strftime('%Y-%m-%d')

'2021-01-06'

In [111]:
sample.to_csv('sample_features'+today.strftime('%Y-%m-%d')+'.csv')

In [112]:
sample.head()

,source,target,connected,sum_of_papers,sum_of_neighbors,log_secundary_neighbors,lenght_short_path,clustering_index_sum
0,Ioannis M. Rekleitis,Gregory Dudek,1.0,55,25,9.699411,1.0,0.761905
1,Oscar Gustafsson,Lars Wanhammar,1.0,17,10,5.894403,1.0,1.178571
2,Mark A. Franklin,Tienyo Pan,1.0,22,7,4.682131,1.0,0.133333
3,Judith Baur,Doug Arnold,1.0,15,27,10.509714,1.0,0.985281
4,Jürgen Weese,Derek L. G. Hill,1.0,72,104,14.214428,1.0,0.710248
